# Task-specific knowledge distillation for Longformer-Encoder-Decoder (LED)
### Abstractive Summarization using Longformer-Encoder-Decoder (LED) as Teacher and ??? as Student
#### Group 22: Anh Le, Kasra Sohrab, and Katie Krupczak

! INTRO HERE !

## Installation

In [12]:
%pip install tensorflow transformers datasets pandas matplotlib --upgrade


  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/58/70/e8ac764ec80810eefcbab0cb1d21dbba6cf26719c44cd6d9a5e9f0407935/tensorflow-2.16.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached tensorflow-2.16.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.3 kB)
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/fc/a5/4d82be566f069d7a9a702dcdf6f9106df0e0b042e738043c0cc7ddd7e3f6/pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/80/3b/e363612ac1a514abfb5505aa209dd5b724b3232a6de98710d7759559706a/matplotlib-3.8.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36

In [13]:
%pip install accelerate -U
%pip install transformers[torch]
%pip install prettytable

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt

2024-04-24 11:34:15.307674: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 11:34:15.849535: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Setup & Configuration


In [11]:
student_id = ""
teacher_id = "allenai/led-base-16384" #Doublecheck

Below are some checks to make sure the `Teacher` & `Student` are creating the same output.

In [15]:
from transformers import AutoTokenizer

# init tokenizer
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_id)
#student_tokenizer = AutoTokenizer.from_pretrained(student_id)

# sample input
sample = "This is a basic example, with different words to test."

# assert results
#assert teacher_tokenizer(sample) == student_tokenizer(sample), "Tokenizers haven't created the same output"

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

OSError: Incorrect path_or_model_id: ''. Please provide either the path to a local folder or the repo_id of a model on the Hub.

## Dataset & Pre-processing
We will be using the ["scientific_papers"](https://huggingface.co/datasets/scientific_papers) dataset available from Hugging Face. This dataset comprises two sets of scientific papers obtained from ArXic and PubMed OpenAccess repositories.

This dataset is ideal for our purposes due to its alignment with LED’s design strengths–processing lengthy sequences.

In [10]:
from datasets import load_dataset

#two options for datasets within scientific_papers: arxiv and pubmed
dataset = load_dataset("scientific_papers", 'pubmed')
dataset
#print(dataset['train'][0]) 

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 119924
    })
    validation: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 6633
    })
    test: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 6658
    })
})

### Pre-processing & Tokenization

To distill our model we need to convert our "Natural Language" to token IDs. We are going to use the tokenizer of the `Teacher`, but since both are creating same output you could also go with the `Student` tokenizer.

### Data Cleaning Methods


In [ ]:
### Data Cleaning and Tokenizing

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(teacher_id)

In [17]:
def preprocess_function(examples):
    # Adjust fields if necessary
    inputs = tokenizer(examples['article'], max_length=16384,truncation=True, padding="max_length")
    targets = tokenizer(examples['abstract'], max_length=512, truncation=True, padding="max_length")
    return {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask'], 'labels': targets['input_ids']}

tokenized_datasets = dataset.map(preprocess_function, batched=True)


tokenized_datasets = dataset.map(preprocess_function, batched=True)

tokenized_datasets["test"].features

Map:   0%|          | 0/119924 [00:00<?, ? examples/s]

Map:   0%|          | 0/6633 [00:00<?, ? examples/s]

Map:   0%|          | 0/6658 [00:00<?, ? examples/s]

Map:   0%|          | 0/119924 [00:00<?, ? examples/s]

{'article': Value(dtype='string', id=None),
 'abstract': Value(dtype='string', id=None),
 'section_names': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [ ]:
## Vocabulary Building